In [2]:
import pandas as pd
import pprint
import spacy
from spacy import displacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk import ngrams
from collections import Counter

In [3]:
#pip install pprint

In [4]:
def word_frequency(sentence):
    sentence =" ".join(sentence)
    new_tokens = word_tokenize(sentence)
    new_tokens = [t.lower() for t in new_tokens]
    new_tokens =[t for t in new_tokens if t not in stopwords.words('english')]
    new_tokens = [t for t in new_tokens if t.isalpha()]
    lemmatizer = WordNetLemmatizer()
    new_tokens =[lemmatizer.lemmatize(t) for t in new_tokens]
    counted = Counter(new_tokens)
    counted_2= Counter(ngrams(new_tokens,2))
    counted_3= Counter(ngrams(new_tokens,3))
    word_freq = pd.DataFrame(counted.items(),columns=['word','frequency']).sort_values(by='frequency',ascending=False)
    word_pairs =pd.DataFrame(counted_2.items(),columns=['pairs','frequency']).sort_values(by='frequency',ascending=False)
    trigrams =pd.DataFrame(counted_3.items(),columns=['trigrams','frequency']).sort_values(by='frequency',ascending=False)
    return word_freq,word_pairs,trigrams

In [52]:
nlp = spacy.load('en_core_web_sm')


C:\Users\christian\miniconda3\lib\site-packages\spacy\util.py:730: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.1.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [114]:
#Traigo Datos
df_tweets = pd.read_csv('exportTweetData.csv')
df_market = pd.read_csv('exportMarketData.csv')

#Combino para juntar todos los twitts en una fila por minuto
dataSetCombinadoTwitter = df_tweets.merge(df_market, how='inner', on='Date')
group = dataSetCombinadoTwitter.groupby('Date')['Tweets'].unique()
df_final =  dataSetCombinadoTwitter.merge(group, how='inner', on='Date')

#Limpio el DataFrame de repetidos e info no utilizada
tweets = df_final['Tweets_y']
textoTweets = []

df_final = df_final.drop_duplicates(subset=['Date'], keep='last')
df_final = df_final.drop(columns=['Tweets_y', 'Tweets_x','Subjectivity','Polarity','Volume'])
df_final['Tweets']=tweets
df_final = df_final.reset_index(drop=True)
for i in range(len(df_final.index)):
    text = '-'.join(df_final['Tweets'].iloc[i])
    textoTweets.append(text)

df_final['Tweets']=textoTweets


In [115]:
#Analisis de Word Frequency y ngramas
#Junto todo el texto de cada fila y una vez hecho eso lo agrego al dataframe
uword,bword,tword = [],[],[]
ufreq, bfreq, tfreq = [],[],[]
for i in range(len(df_final.index)):
    textTweets = "".join(df_final['Tweets'].iloc[i])
    wordsT = word_tokenize(textTweets)
    unigramT,bigramT,trigramT = word_frequency(wordsT)
    uword.append(unigramT['word'].iloc[0])
    ufreq.append(unigramT['frequency'].iloc[0])
    #bigramas
    bword.append(bigramT['pairs'].iloc[0])
    bfreq.append(bigramT['frequency'].iloc[0])
    #trigramas
    tword.append(trigramT['trigrams'].iloc[0])
    tfreq.append(trigramT['frequency'].iloc[0])
    #df_final['unigram'].iloc[i] = unigramT
    
formatoDataSetNgramas = {'Unigrama': uword,'FreqU': ufreq, 'Bigrama':bword, 'FreqB':bfreq, 'Trigrama':tword, 'FreqT':tfreq}
df_ngramas = pd.DataFrame(data=formatoDataSetNgramas)
df_concatenado = pd.concat([df_final, df_ngramas], axis=1)
#df_concatenado

In [ ]:
doc = nlp(textTweets)
    for ent in doc.ents:
        entities.append(ent)
        labels.append(ent.label_)
    columnaEnNombre = 'Entidades'+str(i)
    columnaEtNombre = 'Etiqueta'+str(i)
    df = pd.DataFrame({columnaEnNombre:entities, columnaEtNombre:labels})
    df_NER = pd.concat([df_NER, df], axis = 1)
df_NER = df_NER.dropna()
df_NER    

In [124]:
#Analisis de NER. Me voy a quedar con la entidad mas repetida y su label
allEnt, entFin, allLabls, lablsFin = [],[],[],[]
df_NER = pd.DataFrame()
for i in range(len(df_concatenado.index)):
    entities, labels = [],[]
    textTweets=""
    textTweets = "".join(df_final['Tweets'].iloc[i])
    doc = nlp(textTweets)
    for ent in doc.ents:
        entities.append(ent)
        labels.append(ent.label_)
    columnaEnNombre = 'Entidades'+str(i)
    columnaEtNombre = 'Etiqueta'+str(i)
    df = pd.DataFrame({columnaEnNombre:entities, columnaEtNombre:labels})
    df_NER = pd.concat([df_NER, df], axis = 1)
df_NER = df_NER.dropna()
df_NER    

,Entidades0,Etiqueta0,Entidades1,Etiqueta1,Entidades2,Etiqueta2,Entidades3,Etiqueta3,Entidades4,Etiqueta4,...,Entidades53,Etiqueta53,Entidades54,Etiqueta54,Entidades55,Etiqueta55,Entidades56,Etiqueta56,Entidades57,Etiqueta57
0,"(the, past, , hours)",TIME,"($, QNT, $)",MONEY,(BTC),ORG,"($, xrp, $)",MONEY,"(COMP, /, USDT)",ORG,...,(tomorrow),DATE,(ETH),MONEY,(Matic),NORP,"(’m, not, Hodling, $, BTC)",ORG,"(ONNext, day)",DATE
1,(ICP),MONEY,(BTC),ORG,(BTC),ORG,(BTC),ORG,(COMP),ORG,...,"(days, %)",PERCENT,(BTC),ORG,(BTC),ORG,"($, BTC, $, ETH, $)",MONEY,(first),ORDINAL
2,(ETH),MONEY,"(XRP, $, ALBT, $)",ORG,(DOGE),MONEY,"($, BTC, $, ETH, $)",MONEY,(COMP),MONEY,...,(BTC),ORG,(Crypto),ORG,"($, ETH, $)",MONEY,(BNB),ORG,(-A),ORG
3,($),CARDINAL,(DAG),ORG,(BTCDEC),ORG,(ADA),ORG,"($, USDT$FTX, $)",MONEY,...,(Gutsy),PERSON,(Bitcoin),PERSON,"(MATIC, -, )",PERSON,(BTC),ORG,(BTC),MONEY
4,(BTC),ORG,(ZCX),ORG,(BTC),ORG,($),CARDINAL,"($, BTC, $, ETH, $)",MONEY,...,"(K, $, BTC)",ORG,"($, xrp, $)",MONEY,"(Antminer, Ss)",PERSON,"(Price, Drop)",PERSON,"(this, morning)",TIME
5,"($, ETH, $)",MONEY,"($, VET, $, DOGE, $, XLM, $)",MONEY,(Smart),ORG,"(CNBC, %)",ORG,(BTC),ORG,...,(WEEKEND),ORG,"($, link, $, UNI, $)",MONEY,"(Th, /, kW)",ORG,"(Last, Year)",DATE,(BTC),PRODUCT
